# Task 4.1 - Variational Auto-Encoder

The first part of the task requires the construction of a Variational Auto-Encoder (VAE) for magnetic resonance (MR) images of the brain via the Preprocessed OASIS Dataset. The OASIS dataset was retrieved from /home/groups/comp3710/ directory on the Rangpur cluster.

Full marks for this section require the training of the model on the dataset and the visualisation of the resulting manifold output from the VAE. Visualisation can be done using sampling methods to create a 2D image, or using a dimensionality reduction technique such as a UMAP.

In [ ]:
# Code block